Original implementation of Contrastive-sc method
(https://github.com/ciortanmadalina/contrastive-sc)

In [1]:
import sys
sys.path.append("..")
import argparse
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from sklearn.cluster import KMeans
from sklearn import metrics

import torch
import torch.nn as nn
import copy
from tqdm.notebook import tqdm
import models
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import st_loss
import time
import h5py
import scipy as sp
import scanpy.api as sc
from collections import Counter
import random
import utils
import loop
import pickle

import train
import os
import glob2
plt.ion()
plt.show()
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

/opt/conda/lib/python3.7/site-packages/scanpy/api/__init__.py:7: FutureWarning: 

In a future version of Scanpy, `scanpy.api` will be removed.
Simply use `import scanpy as sc` and `import scanpy.external as sce` instead.

  FutureWarning,


In [2]:
path = "../"
category = "real_data"
files = glob2.glob(f'{path}{category}/*.h5')
files = [f[len(f"'{path}{category}"):-3] for f in files]
files

['Quake_Smart-seq2_Trachea',
 'Quake_Smart-seq2_Diaphragm',
 'Quake_10x_Spleen',
 'Young',
 'mouse_ES_cell',
 'Adam',
 'Quake_10x_Bladder',
 'Quake_Smart-seq2_Lung',
 'Quake_10x_Limb_Muscle',
 'worm_neuron_cell',
 'mouse_bladder_cell',
 'Romanov',
 'Quake_Smart-seq2_Limb_Muscle',
 'Muraro',
 '10X_PBMC']

In [3]:
sczi = pd.read_pickle(f"../output/pickle_results/real_data/real_data_sczi.pkl")
# load the grid search result having the optimal nb of genes per dataset
optimal = pd.read_pickle(f"{path}output/pickle_results/real_data/optimal_input_size.pkl")
optimal.head()

,dataset,nb_genes
0,Quake_Smart-seq2_Trachea,500
1,Quake_Smart-seq2_Diaphragm,1000
2,Quake_10x_Spleen,500
3,Young,1000
4,mouse_ES_cell,1000


In [4]:
df = pd.DataFrame()
dropout = 0.9
lr = 0.4
layers = [200, 40, 60]
temperature = 0.07
for dataset in files:

    print(f">>>>> Data {dataset}")
    print("SCZI ", sczi[sczi["dataset"] == dataset]["ARI"].mean())
    t0 = time.time()
    data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
    X = np.array(data_mat['X'])
    Y = np.array(data_mat['Y'])

    cluster_number = np.unique(Y).shape[0]
    nb_genes = optimal[optimal["dataset"] == dataset]["nb_genes"].values[0]
    data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
    X = np.array(data_mat['X'])
    Y = np.array(data_mat['Y'])

    X = train.preprocess(X, nb_genes=nb_genes)
    preprocess_time = time.time()-t0
    for run in range(3):
        torch.manual_seed(run)
        torch.cuda.manual_seed_all(run)
        np.random.seed(run)
        random.seed(run)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        start = time.time()
        dresults = train.run(X,
                             cluster_number,
                             dataset,
                             Y=Y,
                             nb_epochs=30,
                             lr=lr,
                             temperature=temperature,
                             dropout=dropout,
                             layers=layers,
                             save_to=f"{path}output/{category}/{run}/",
                             save_pred = True)
        time_k = (dresults["t_k"] - start) + preprocess_time
        time_l = (dresults["t_clust"] - start) + (dresults["t_l"] -dresults["t_k"]) +preprocess_time
        dresults["temperature"] = temperature
        dresults["dropout"] = dropout
        dresults["nb_genes"] = nb_genes
        dresults["layers"] = str(layers)
        dresults["run"] = run
        dresults["t_k"] = time_k
        dresults["t_l"] = time_l
        df = df.append(dresults, ignore_index=True)

        df.to_pickle(f"{path}output/pickle_results/{category}/{category}_baseline.pkl")

>>>>> Data Quake_Smart-seq2_Trachea
SCZI  0.8258284521205065
(1350, 23341) (1350, 23341) keeping 500 genes
>>>>> Data Quake_Smart-seq2_Diaphragm
SCZI  0.9596701189611787
(870, 23341) (870, 23341) keeping 1000 genes
>>>>> Data Quake_10x_Spleen
SCZI  0.9207243162607069
(9552, 23341) (9552, 23341) keeping 500 genes
>>>>> Data Young
SCZI  0.6966379900869195
(5685, 33658) (5685, 33658) keeping 1000 genes
>>>>> Data mouse_ES_cell
SCZI  0.8004502714264402
(2717, 24175) (2717, 24175) keeping 1000 genes
>>>>> Data Adam
SCZI  0.8632277666545504
(3660, 23797) (3660, 23797) keeping 500 genes
>>>>> Data Quake_10x_Bladder
SCZI  0.9828247498197693
(2500, 23341) (2500, 23341) keeping 500 genes
>>>>> Data Quake_Smart-seq2_Lung
SCZI  0.754207987444416
(1676, 23341) (1676, 23341) keeping 1000 genes
>>>>> Data Quake_10x_Limb_Muscle
SCZI  0.9609079013375856
(3909, 23341) (3909, 23341) keeping 500 genes
>>>>> Data worm_neuron_cell
SCZI  0.06355801295995496
(4186, 13488) (4186, 13488) keeping 5000 genes
>>>>

In [6]:
df.mean() #0.347519, # 0.499074# 0.520911

dropout        9.000000e-01
kmeans_ari     7.717891e-01
kmeans_cal     4.714539e+03
kmeans_nmi     8.052522e-01
kmeans_sil     5.983083e-01
leiden_ari     5.135286e-01
leiden_cal     4.106211e+03
leiden_nmi     7.087917e-01
leiden_sil     3.971199e-01
nb_genes       1.066667e+03
run            1.000000e+00
t_clust        1.613319e+09
t_k            5.382012e+00
t_l            1.340280e+01
temperature    7.000000e-02
time           2.169566e+00
dtype: float64

# Train on cpu

In [ ]:
df = pd.DataFrame()
dropout = 0.9
lr = 0.4
layers = [200, 40, 60]
temperature = 0.07
for dataset in files:

    print(f">>>>> Data {dataset}")
    print("SCZI ", sczi[sczi["dataset"] == dataset]["ARI"].mean())
    t0 = time.time()
    data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
    X = np.array(data_mat['X'])
    Y = np.array(data_mat['Y'])

    cluster_number = np.unique(Y).shape[0]
    nb_genes = optimal[optimal["dataset"] == dataset]["nb_genes"].values[0]
    data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
    X = np.array(data_mat['X'])
    Y = np.array(data_mat['Y'])

    X = train.preprocess(X, nb_genes=nb_genes)
    preprocess_time = time.time()-t0
    for run in range(3):
        torch.manual_seed(run)
        torch.cuda.manual_seed_all(run)
        np.random.seed(run)
        random.seed(run)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        start = time.time()
        dresults = train.run(X,
                             cluster_number,
                             dataset,
                             Y=Y,
                             nb_epochs=30,
                             lr=lr,
                             temperature=temperature,
                             dropout=dropout,
                             layers=layers,
                             save_to=f"{path}output/{category}/{run}/",
                             save_pred = False,
                             use_cpu= True
                            )
        time_k = (dresults["t_k"] - start) + preprocess_time
        time_l = (dresults["t_clust"] - start) + (dresults["t_l"] -dresults["t_k"]) +preprocess_time
        dresults["temperature"] = temperature
        dresults["dropout"] = dropout
        dresults["nb_genes"] = nb_genes
        dresults["layers"] = str(layers)
        dresults["run"] = run
        dresults["t_k"] = time_k
        dresults["t_l"] = time_l
        df = df.append(dresults, ignore_index=True)

        df.to_pickle(f"{path}output/pickle_results/{category}/{category}_baseline_cpu.pkl")

In [ ]:
df.mean()

# Debug

In [ ]:
dataset = 'Quake_Smart-seq2_Diaphragm'

In [ ]:
x =df[df["dataset"] == dataset]
x

In [ ]:
data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
X = np.array(data_mat['X'])
Y = np.array(data_mat['Y'])
X.shape, Y.shape

In [ ]:

cluster_number = np.unique(Y).shape[0]
cluster_number

In [ ]:
f = x["features_0"].values[0]

In [ ]:
kmeans = KMeans(n_clusters=cluster_number,
                init="k-means++",
                random_state=0)
pred = kmeans.fit_predict(f)

In [ ]:
adjusted_rand_score(Y, pred)

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
X_embedded = TSNE(n_components=2).fit_transform(f)

In [ ]:
n_neighbors=200
n_pcs=0

adata = sc.AnnData(f)
sc.pp.neighbors(adata, n_neighbors=n_neighbors, n_pcs=n_pcs, use_rep='X'
           )
sc.tl.leiden(adata)
pred = adata.obs['leiden'].to_list()
pred = [int(x) for x in pred]
print(np.unique(pred).shape)
adjusted_rand_score(Y, pred)

In [ ]:
plt.scatter(X_embedded[:, 0], X_embedded[:, 1], c = pred)